In [1]:
import os
os.chdir('..')
import sys
sys.path.insert(0, "/Users/ludo/Desktop/animalai/animalai/animalai_train")
sys.path.insert(1, "/Users/ludo/Desktop/animalai/animalai/animalai")

from raw_learned import Pipeline
from collections import namedtuple
from animalai.envs.arena_config import ArenaConfig
import os
margs = namedtuple('args', 'env seed arena_config num_episodes inference')
env_path = '../env/aaiagain'
ac = "configurations/curriculum4/35.yml"
ac = [ArenaConfig(ac)]
seed = 3
# ac = [ArenaConfig(f"../competition_configurations/{i}.yml") for i in x]
# print(f"Total number of arenas:{len(ac)}")
# args = margs(env=env_path, arena_config=ac, num_episodes=len(ac), inference=True)
args = margs(env=env_path, seed=seed, arena_config=ac, num_episodes=120, inference=False)
pipe = Pipeline(args)

USING LOCAL LIBRARY


INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
INFO:mlagents_envs:Connected new brain:
AnimalAI?team=0
INFO:gym_unity:1 agents within environment.


In [2]:
import time
start = time.time()
x = pipe.learn_run()
end = time.time()
print(f"Total time taken: {end-start}")


NEW RULES LEARNED: initiate(interact(V0),V2) :- goal(V0), visible(V0, V1, V2).

NEW RULES LEARNED: initiate(explore(V0,V3,V1),V2) :- visible(V0, V1, V2), occludes(V0,V3, V2).

NO RULES LEARNED
Final results: 102/120 episodes were completed successfully
Total time taken: 303.5445759296417


In [1]:
list(range(1,2))

[1]